# Greedy Technique

Given an array of size n that contain either ‘G’rab car or ‘P’assenger. 

Each Grab car can pick up only one passenger. And each Grab car cannot pick up a passenger who is more than K units away from Grab car.

1. Write a program using Brute force approach to find number of all solutions 
    that give the maximum number of Passenger(s) that can ride Grab(s).
2. Write a program using Greedy Technique to find a solution that gives the maximum number of Passengers(s) that can ride Grab(s).

For example, if an array consists of {‘G’, ‘P’, ‘P’, ‘G’, ‘P’} and we set k = 1, 

then the output the maximum number passenger can ride Grab would be 2. 

The first Grab picks up the first passenger and the second Grab picks up either the second or third passenger.

---

## Reading test case

In [127]:
def read_input(filename):
    with open(filename, 'r') as f:
        arr = list(f.readline().strip())
        k = int(f.readline().strip())
    return arr, k

# The functions 'can_pickup', 'recursive_search' and 'find_max_pickups' remain unchanged from the previous answer

# Reading the input
filename = 'test.txt'
arr, k = read_input(filename)
print(arr,"\n",k)

['G', 'P', 'G', 'P', 'G', 'P', 'G', 'P', 'G', 'P', 'G', 'P', 'G', 'P', 'G', 'P', 'G', 'P', 'G', 'P'] 
 2


### Brute force

In [126]:
def can_pickup(g_index, p_index, k):
    """Checks if a Grab car at g_index can pick up a passenger at p_index"""
    return abs(g_index - p_index) <= k

def brute_force(arr, k, start=0, pickups=[]):
    """Recursively find all possible pairings of Grab cars and passengers"""
    n = len(arr)
    max_pickups = len(pickups)
    best_solutions = [pickups]

    for i in range(start, n):
        if arr[i] == 'G':
            for j in range(n):
                if arr[j] == 'P' and not any(p[1] == j for p in pickups) and can_pickup(i, j, k):
                    new_pickups, new_solutions = find_pairings(arr, k, i + 1, pickups + [(i, j)])
                    if new_pickups == max_pickups:
                        best_solutions.extend(new_solutions)
                    elif new_pickups > max_pickups:
                        max_pickups = new_pickups
                        best_solutions = new_solutions

    return max_pickups, best_solutions

max_passengers, solutions = brute_force(arr, k)

print(f"Maximum passengers: {max_passengers}")
for idx, solution in enumerate(solutions, 1):
    print(f"Solution {idx}: {solution}")


Maximum passengers: 10
Solution 1: [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11), (12, 13), (14, 15), (16, 17), (18, 19)]


---

### Greedy -> use nearest passenger

In [128]:
def greedy_max_passenger_V2(arr, k):
    n = len(arr)
    paired = {}  # dictionary can provide O(1) lookup time
    pairs = []

    for i in range(n):
        if arr[i] == 'G':
            paired_passenger = None

            # Find the nearest passenger to the left within k distance
            for j in range(i-1, i-1-k, -1):
                if 0 <= j and arr[j] == 'P' and j not in paired:
                    paired_passenger = j
                    break

            # If no passenger was found to the left, find the nearest passenger to the right within k distance
            if paired_passenger is None:
                for j in range(i+1, i+1+k):
                    if j < n and arr[j] == 'P' and j not in paired:
                        paired_passenger = j
                        break

            # If a passenger was found, add to pairs and mark as paired
            if paired_passenger is not None:
                pairs.append((i, paired_passenger))
                paired[paired_passenger] = True

    return len(pairs), pairs

max_passengers, solution = greedy_max_passenger_V2(arr, k)

print(f"Maximum passengers: {max_passengers}")
print(f"Solution: {solution}")

Maximum passengers: 10
Solution: [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11), (12, 13), (14, 15), (16, 17), (18, 19)]


---

### What if we use nearest passenger within the k distance

In [129]:
def greedy_max_passenger_V2(arr, k):
    n = len(arr)
    paired = {}  # dictionary can provide O(1) lookup time
    pairs = []

    for i in range(n):
        if arr[i] == 'G':
            paired_passenger = None

            # Find the farthest passenger to the left within k distance
            for j in range(i-k, i):
                if 0 <= j and arr[j] == 'P' and j not in paired:
                    paired_passenger = j
                    break

            # If no passenger was found to the left, find the farthest passenger to the right within k distance
            if paired_passenger is None:
                for j in range(i+k, i,-1):
                    if j < n and arr[j] == 'P' and j not in paired:
                        paired_passenger = j
                        break

            # If a passenger was found, add to pairs and mark as paired
            if paired_passenger is not None:
                pairs.append((i, paired_passenger))
                paired[paired_passenger] = True

    return len(pairs), pairs

max_passengers, solution = greedy_max_passenger_V2(arr, k)

print(f"Maximum passengers: {max_passengers}")
print(f"Solution: {solution}")

Maximum passengers: 10
Solution: [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11), (12, 13), (14, 15), (16, 17), (18, 19)]
